In [37]:
from explore.smartframe import bioframe
from joblib import Parallel, delayed
import multiprocessing
import csv

from datetime import datetime

import pandas as pd
from scipy.stats import kurtosis


class sketch:
    def uint_sketch(self, paths, uid, tw="300s", var='temp_skin', prefix_dir="/cephfs/tempredict/sketch/",
                    fillin=True):
        try:
            b = bioframe()
            df, sql = b.load(paths, uid=uid)
            df["timestamp_utc_dt"] = df.apply(lambda x: self.local_time(x["timestamp_utc"]), axis=1)

            tf = []
            tf.append("timestamp_utc_dt")
            b.add_local_time(df, tf, utc='timezone_offset_minutes')

            df3 = df.drop(
                columns=['timezone_offset_minutes', 'participant_id', 'file_format', 'data_type', 'timestamp_utc',
                         'timestamp_utc_dt', 'study_id'])
            # p_inx = p_inx_t.compute()
            print(df3.head(10))
            df2 = df3.compute()

            p_inxl = df2.set_index(['timestamp_utc_dt_local'])
            sktch_agg = None
            p_sample = p_inxl.resample(tw)

            if fillin:
                pskt = p_sample.agg({kurtosis, 'std', 'mean', 'skew', 'var', 'count', 'min', 'max'})
            else:
                pskt = p_sample.agg({kurtosis, 'std', 'mean', 'skew', 'var', 'count', 'min', 'max'})
            if pskt is not None:
                quantile_10 = p_sample.quantile(0.10)
                quantile_20 = p_sample.quantile(0.20)
                quantile_25 = p_sample.quantile(0.25)
                quantile_30 = p_sample.quantile(0.30)
                quantile_40 = p_sample.quantile(0.40)
                quantile_50 = p_sample.quantile(0.50)
                quantile_60 = p_sample.quantile(0.60)
                quantile_70 = p_sample.quantile(0.70)
                quantile_75 = p_sample.quantile(0.75)
                quantile_80 = p_sample.quantile(0.80)
                quantile_90 = p_sample.quantile(0.90)
                # print("*********")
                # if quantile_10 is not None:
                #     print(quantile_10[var])
                #     print("$$$$$")
                try:
                    pskt['10_per'] = quantile_10[var]
                except Exception as e:
                    pskt['10_per'] = "NaN"
                try:
                    pskt['20_per'] = quantile_20[var]
                except Exception as e:
                    pskt['20_per'] = "NaN" 
                try:
                    pskt['30_per'] = quantile_30[var]
                except Exception as e:
                    pskt['30_per'] = "NaN"   
                try:
                    pskt['40_per'] = quantile_40[var]
                except Exception as e:
                    pskt['40_per'] = "NaN"  
                try:
                    pskt['50_per'] = quantile_50[var]
                except Exception as e:
                    pskt['50_per'] = "NaN"  
                try:
                    pskt['60_per'] = quantile_60[var]
                except Exception as e:
                    pskt['60_per'] = "NaN"  
                try:
                    pskt['70_per'] = quantile_70[var]
                except Exception as e:
                    pskt['70_per'] = "NaN"  
                try:
                    pskt['80_per'] = quantile_80[var]
                except Exception as e:
                    pskt['80_per'] = "NaN"   
                try:
                    pskt['90_per'] = quantile_90[var]
                except Exception as e:
                    pskt['90_per'] = "NaN"                            
                # print(quantile_10[var])
                # pskt['20_per'] = quantile_20[var]
                # pskt['25_per'] = quantile_25[var]
                # pskt['30_per'] = quantile_30[var]
                # pskt['40_per'] = quantile_40[var]
                # pskt['50_per'] = quantile_50[var]
                # pskt['60_per'] = quantile_60[var]
                # pskt['70_per'] = quantile_70[var]
                # pskt['75_per'] = quantile_75[var]
                # pskt['80_per'] = quantile_80[var]
                # pskt['90_per'] = quantile_90[var]
            pskt['pid'] = uid
            # print(pskt)
            pskt.to_csv(prefix_dir + str(var) + "/" + uid + ".csv", sep=',')

            return pskt;
        except Exception as e:
            print("error: after writing", e.args, var)

    def local_time(self, x):
        y = 0
        try:
            y = pd.to_datetime(datetime.fromtimestamp(x).strftime('%Y-%m-%d-%H:%M:%S'))
        except Exception as e:
            y = datetime.strptime("1500-12-31 21:19:00 +00:00", '%Y-%m-%d %H:%M:%S %z')
        return y


num_cores = multiprocessing.cpu_count()
print(num_cores)
path_index = "/datavol/tempredict/index/temppredict/postgres_public_full_catalog_hrv.csv"
with open(path_index, mode='r') as infile:
    reader = csv.reader(infile)
    dcp = "/datavol/tempredict/org_data_don_use/study_id=71c9af3d-1c57-4e8e-96fb-88b10ee9acb8/"
    mydict = {}
    i = 0
    for rows in reader:
        i = i + 1
        mydict[rows[1]] = dcp + str(rows[0])


def split_dictionary(input_dict, chunk_size):
    res = []
    new_dict = {}
    for k, v in input_dict.items():
        if len(new_dict) < chunk_size:
            new_dict[k] = v
        else:
            res.append(new_dict)
            new_dict = {k: v}
    res.append(new_dict)
    return res

dicts = split_dictionary(mydict, num_cores)
# print(len(dicts))
def uprocess(mydict):
    for key, value in mydict.items():
        # print("1."+str(value))
        t = sketch()
        try:
            t.uint_sketch(value, key, tw="1800s", var="hrv",prefix_dir="/datavol/tempredict/sketch/30-min/")
        except Exception as e:
            print("error:: upeer uprocess", e.args)

256


In [38]:
path="/datavol/tempredict/org_data_don_use/study_id=71c9af3d-1c57-4e8e-96fb-88b10ee9acb8/full_history/data_type=hrv_5min/file_format=parquet/participant_id=cc165e05-c300-492c-bf25-7d8d8e839696/hrv_5min.snappy.parquet"
uid="JciY-wUFkrdSyI1UcP15Uk1AlB3de-x6i6Oa1avgxikOpxx15Wvr-BZjvL84dy4ca8Q"
t = sketch()
t.uint_sketch(path, uid, tw="1800s", var="hrv",prefix_dir="/datavol/tempredict/sketch/30-min/")


/home/docker/.local/lib/python3.10/site-packages/dask/dataframe/core.py:5727: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))
/home/docker/.local/lib/python3.10/site-packages/dask/dataframe/core.py:5727: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'datetime64[ns]'))



(Delayed('int-0aff96d2-cc5d-47d2-96f9-b7075e8acd62'), 8)
       HR  rMSSD timestamp_utc_dt_local
33950  74     20    2020-01-01 02:18:15
33951  75     16    2020-01-01 02:23:15
33952  74     15    2020-01-01 02:28:15
33953  73     19    2020-01-01 02:33:15
33954  73     17    2020-01-01 02:38:15
33955  76     15    2020-01-01 02:43:15
33956  76     16    2020-01-01 02:48:15
33957  76     15    2020-01-01 02:53:15
33958  78     15    2020-01-01 02:58:15
33959  78     15    2020-01-01 03:03:15


/home/docker/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1417: RuntimeWarning: Mean of empty slice.
  mean = a.mean(axis, keepdims=True)
/home/docker/.local/lib/python3.10/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/docker/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/docker/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/docker/.local/lib/python3.10/site-packages/pandas/core/groupby/groupby.py:1668: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  f = lambda x: func(x, *args, **kwargs)
/home/docker/.local/lib/python3.10/site-packages/scip

HR                                           \
                             std       var  kurtosis      skew       mean   
timestamp_utc_dt_local                                                      
2020-01-01 02:00:00     0.577350  0.333333 -1.500000  1.732051  74.333333   
2020-01-01 02:30:00     1.966384  3.866667 -1.227111 -0.219202  75.333333   
2020-01-01 03:00:00     0.547723  0.300000 -2.000000  0.000000  77.500000   
2020-01-01 03:30:00     1.211060  1.466667 -1.388430  0.075066  78.333333   
2020-01-01 04:00:00     0.516398  0.266667 -1.500000  0.968246  77.333333   
...                          ...       ...       ...       ...        ...   
2020-08-31 07:30:00          NaN       NaN       NaN       NaN        NaN   
2020-08-31 08:00:00          NaN       NaN       NaN       NaN        NaN   
2020-08-31 08:30:00          NaN       NaN       NaN       NaN        NaN   
2020-08-31 09:00:00          NaN       NaN       NaN       NaN        NaN   
2020-08-31 09:30:00          NaN       NaN       NaN       NaN  65.000000   

                                             rMSSD            ... 10_per  \
                         max count   min       std       var  ...          
timestamp_utc_dt_local                                        ...          
2020-01-01 02:00:00     75.0     3  74.0  2.645751  7.000000  ...    NaN   
2020-01-01 02:30:00     78.0     6  73.0  1.602082  2.566667  ...    NaN   
2020-01-01 03:00:00     78.0     6  77.0  2.366432  5.600000  ...    NaN   
2020-01-01 03:30:00     80.0     6  77.0  0.816497  0.666667  ...    NaN   
2020-01-01 04:00:00     78.0     6  77.0  2.529822  6.400000  ...    NaN   
...                      ...   ...   ...       ...       ...  ...    ...   
2020-08-31 07:30:00      NaN     0   NaN       NaN       NaN  ...    NaN   
2020-08-31 08:00:00      NaN     0   NaN       NaN       NaN  ...    NaN   
2020-08-31 08:30:00      NaN     0   NaN       NaN       NaN  ...    NaN   
2020-08-31 09:00:00      NaN     0   NaN       NaN       NaN  ...    NaN   
2020-08-31 09:30:00     65.0     1  65.0       NaN       NaN  ...    NaN   

                       20_per 30_per 40_per 50_per 60_per 70_per 80_per  \
                                                                          
timestamp_utc_dt_local                                                    
2020-01-01 02:00:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2020-01-01 02:30:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2020-01-01 03:00:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2020-01-01 03:30:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2020-01-01 04:00:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...                       ...    ...    ...    ...    ...    ...    ...   
2020-08-31 07:30:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2020-08-31 08:00:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2020-08-31 08:30:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2020-08-31 09:00:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2020-08-31 09:30:00       NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                       90_per  \
                                
timestamp_utc_dt_local          
2020-01-01 02:00:00       NaN   
2020-01-01 02:30:00       NaN   
2020-01-01 03:00:00       NaN   
2020-01-01 03:30:00       NaN   
2020-01-01 04:00:00       NaN   
...                       ...   
2020-08-31 07:30:00       NaN   
2020-08-31 08:00:00       NaN   
2020-08-31 08:30:00       NaN   
2020-08-31 09:00:00       NaN   
2020-08-31 09:30:00       NaN   

                                                                      pid  
                                                                           
timestamp_utc_dt_local                                                     
2020-01-01 02:00:00     JciY-wUFkrdSyI1UcP15Uk1AlB3de-x6i6Oa1avgxikOpx...  
2020-01-01 02:30:00     JciY-wUFkrdSyI1UcP15Uk1AlB3de-x6i6Oa1avgxikO

In [2]:
for key, value in mydict.items():
        print("1."+str(value))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
mydict

{'pid': '/datavol/tempredict/org_data_don_use/study_id=71c9af3d-1c57-4e8e-96fb-88b10ee9acb8/path',
 'JciY-wUFkrdSyI1UcP15Uk1AlB3de-x6i6Oa1avgxikOpxx15Wvr-BZjvL84dy4ca8Q': '/datavol/tempredict/org_data_don_use/study_id=71c9af3d-1c57-4e8e-96fb-88b10ee9acb8/full_history/data_type=hrv_5min/file_format=parquet/participant_id=cc165e05-c300-492c-bf25-7d8d8e839696/hrv_5min.snappy.parquet',
 'PLd5iOmWiKdOAV06DjWZ_HZDtAo5bjJTL8PKakXNGQXlv3Huc8t0uBYFox5LsK98TwA': '/datavol/tempredict/org_data_don_use/study_id=71c9af3d-1c57-4e8e-96fb-88b10ee9acb8/full_history/data_type=hrv_5min/file_format=parquet/participant_id=cc208c87-23ac-40d6-b87c-b26734f12941/hrv_5min.snappy.parquet',
 '9ezUDYrHM9U78SdiZLF_yXNyX0r3VWEBZBtW9tdhNJ-pZUuRufDkvpppwcAI-kSPaTU': '/datavol/tempredict/org_data_don_use/study_id=71c9af3d-1c57-4e8e-96fb-88b10ee9acb8/full_history/data_type=hrv_5min/file_format=parquet/participant_id=cc2768f5-ffb1-4597-b3ad-9201daa87b4e/hrv_5min.snappy.parquet',
 'whwvj4C4MjGM4Ec7dzsYtffRyANyZ330uBPfYTJk3